In [7]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [8]:
mp_key = "7H4ydwHDAGw2EfDFEGcbuln7Cw6lpJsi"
mp_id = "mp-30459"

In [9]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)

conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Symétrie de  structure du ScNiBi

In [18]:
view = quick_view(prim_struc, "packed", conventional = True)
display(view)



JsmolView(layout=Layout(align_self='stretch', height='400px'))

Positions de chaque atome dans la cellule conventionnelle et primitive

In [19]:
conv_struc.sites

[PeriodicSite: Sc (3.111, 0.0, 1.905e-16) [0.5, 0.0, 0.0],
 PeriodicSite: Sc (3.111, 3.111, 3.111) [0.5, 0.5, 0.5],
 PeriodicSite: Sc (0.0, 0.0, 3.111) [0.0, 0.0, 0.5],
 PeriodicSite: Sc (5.003e-16, 3.111, 1.905e-16) [0.0, 0.5, 0.0],
 PeriodicSite: Ni (4.667, 1.556, 1.556) [0.75, 0.25, 0.25],
 PeriodicSite: Ni (4.667, 4.667, 4.667) [0.75, 0.75, 0.75],
 PeriodicSite: Ni (1.556, 1.556, 4.667) [0.25, 0.25, 0.75],
 PeriodicSite: Ni (1.556, 4.667, 1.556) [0.25, 0.75, 0.25],
 PeriodicSite: Bi (0.0, 0.0, 0.0) [0.0, 0.0, 0.0],
 PeriodicSite: Bi (5.003e-16, 3.111, 3.111) [0.0, 0.5, 0.5],
 PeriodicSite: Bi (3.111, 0.0, 3.111) [0.5, 0.0, 0.5],
 PeriodicSite: Bi (3.111, 3.111, 3.81e-16) [0.5, 0.5, 0.0]]

In [20]:
prim_struc.sites

[PeriodicSite: Sc (2.54, 1.796, 4.4) [0.5, 0.5, 0.5],
 PeriodicSite: Ni (1.27, 0.8982, 2.2) [0.25, 0.25, 0.25],
 PeriodicSite: Bi (0.0, 0.0, 0.0) [-0.0, 0.0, 0.0]]

In [21]:
Sc_pos_init = conv_struc.sites[0].frac_coords
Ni_pos_init = conv_struc.sites[5].frac_coords
Bi_pos_init = conv_struc.sites[9].frac_coords
print(Sc_pos_init)
print(Ni_pos_init)
print(Bi_pos_init)

[0.5 0.  0. ]
[0.75 0.75 0.75]
[0.  0.5 0.5]


In [22]:
symmop_1 = symmops[1] # rot-inversion d'ordre 4 (4-bar)

symmop_2 = symmops[5] # miroir (m)
symmop_3 = symmops[8] # rotation d'ordre 3 (C3)

print(symmop_1.as_xyz_str())
print(symmop_2.as_xyz_str())
print(symmop_3.as_xyz_str())


y, -x, -z
y, x, z
z, x, y


In [23]:
Sc_pos_final = symmop_1.operate(Sc_pos_init)
Ni_pos_final = symmop_2.operate(Ni_pos_init)
Bi_pos_final = symmop_3.operate(Bi_pos_init)
print(Sc_pos_final)
print(Ni_pos_final)
print(Bi_pos_final)

[ 0.  -0.5  0. ]
[0.75 0.75 0.75]
[0.5 0.  0.5]


## Axe de roto-inversion d'ordre 4

In [25]:
print(f"Les coordonnées initial de l'atome de Sc sont: {Sc_pos_init}\n")
print(f"l'équation de la symétrie étant : {symmop_1.as_xyz_str()},")
print(f"la matrices de rotation et le vecteur de translation sont : \n {symmop_1} \n")
print("Après l'opération de symmétrie, les coordonnées de Sc sont : ",Sc_pos_final, "\n")


view_Sc = quick_view(prim_struc, "packed", conventional = True)
display(view_Sc)

view_Sc.script('draw SYMOP 2 {atomno = 1}')

Les coordonnées initial de l'atome de Sc sont: [0.5 0.  0. ]

l'équation de la symétrie étant : y, -x, -z,
la matrices de rotation et le vecteur de translation sont : 
 Rot:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.] 

Après l'opération de symmétrie, les coordonnées de Sc sont :  [ 0.  -0.5  0. ] 



JsmolView(layout=Layout(align_self='stretch', height='400px'))

## Plan miroir 

In [26]:
print(f"Les coordonnées initial de l'atome de Ni sont: {Ni_pos_init}\n")
print(f"l'équation de la symétrie étant : {symmop_2.as_xyz_str()},")
print(f"la matrices de rotation et le vecteur de translation sont : \n {symmop_2} \n")
print("Après l'opération de symmétrie, les coordonnées de Ni sont : ",Ni_pos_final, "\n")

view_Ni = quick_view(prim_struc, "packed", conventional = True)
display(view_Ni)



Les coordonnées initial de l'atome de Ni sont: [0.75 0.75 0.75]

l'équation de la symétrie étant : y, x, z,
la matrices de rotation et le vecteur de translation sont : 
 Rot:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
tau
[0. 0. 0.] 

Après l'opération de symmétrie, les coordonnées de Ni sont :  [0.75 0.75 0.75] 



JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [27]:
view_Ni.script('draw SYMOP 6 {atomno = 6}')

## Rotation d'ordre 3

In [28]:
print(f"Les coordonnées initial de l'atome de Bi sont: {Bi_pos_init}\n")
print(f"l'équation de la symétrie étant : {symmop_3.as_xyz_str()},")
print(f"la matrices de rotation et le vecteur de translation sont : \n {symmop_3} \n")
print("Après l'opération de symmétrie, les coordonnées de Bi sont : ",Bi_pos_final, "\n")


view_Bi = quick_view(prim_struc, "packed", conventional = True)
display(view_Bi)



Les coordonnées initial de l'atome de Bi sont: [0.  0.5 0.5]

l'équation de la symétrie étant : z, x, y,
la matrices de rotation et le vecteur de translation sont : 
 Rot:
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
tau
[0. 0. 0.] 

Après l'opération de symmétrie, les coordonnées de Bi sont :  [0.5 0.  0.5] 



JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [29]:
view_Bi.script('draw SYMOP 9 {atomno = 10}')

In [ ]:
view.close()
view_Sc.close()
view_Ni.close()
view_Bi.close()